# Final Capstone Project

## A typical German Café in New York (Brooklyn)

### Introduction/Business Problem

New York is one of the worlds biggest and most famous cities. It is also the hometown of about 8.5 million people and is divided in 5 very different boroughs: the Bronx, Brooklyn, Manhattan, Queens and Staten Island.

A friend of mine wants to open a typical German Café (with e.g. Black Forest Cherry Cake) in New York (Brooklyn). But unfortunately she has no idea which neighbourhoodd suits best. For this reason I offered her to explore and analyse the different neighbourhoods to find out where Cafés and German Restaurants are already located.

### Data

Via Foursquare I will get the location data from New York (including latitude and longitude as well as the venue category). I will analyse and sort the data to find out in which neighbourhoods of Manhattan there are already German Cafés (Cafés and German restaurants). Afterwards I will try to cluster and visualize the data so it is easier to see where Cafés and German restaurants are in demand and can survive.

### Methodology

First, I used and opend the New York data set from the lab, which contains the different boroughs, neighbourhoods and lat/long. Next I filtered these data set to only get Brooklyn data. To get a better feeling I visualized it using folium. 

After this I used Foursquare API to get venue information (top 100, with radius of 500m). With one hot encoding and mean it was easier to see how often you can find each venue in each neighbourhood. Then I created a dataframe with the top ten venues in each neighbourhood. 

This dataframe was the basis for my k means clustering. I tried different amounts of clusters (4, 5, 6, 8 and 10), but finally decided to take 5 clusters. In the end I printed out each cluster with its neighbourhoods and top ten clusters and started exploring them (searching for cafés, bakeries and German restaurants).

### Code

In [1]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  24.27 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  40.33 MB/s


Get and open New York data which contains long, lat, borough (5) and neighbourhood (306)

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [3]:
newyork_data

{'bbox': [-74.2492599487305,
  40.5033187866211,
  -73.7061614990234,
  40.9105606079102],
 'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [-73.84720052054902,
     40.89470517661],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.1',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661],
    'borough': 'Bronx',
    'name': 'Wakefield',
    'stacked': 1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.82993910812398, 40.87429419303012],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.2',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.874294193

In [4]:
#all relevant data is in 'features', therefore we can copy this in a new list
neighborhoods_data = newyork_data['features']

In [5]:
#the next step is to convert these dictionary entries from the list in a pandas dataframe
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [6]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Visualization of NY data with geocoder and folium

In [7]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_exp")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [8]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

map_ny = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  32.95 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.98 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  38.64 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  25.19 MB/s


In [30]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)

In [31]:
# after filtering the data to get only manhattan, we will add the geographical information
address = 'Brooklyn, NY'
geolocator = Nominatim(user_agent="ny_exp")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [32]:
# now let's create a map with folium - as we did for ny data
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)   
map_brooklyn

### Foursquare

In [33]:
CLIENT_ID = '24VAK0CESHDGCRSKKE3D1O3TUC1FH2VWEUIGHCTLZ5BNVH5F' # your Foursquare ID
CLIENT_SECRET = 'DSMRBBWX231WHXZZADRI2D4RYT1TDS033XEM11B2RIZWKWKG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 24VAK0CESHDGCRSKKE3D1O3TUC1FH2VWEUIGHCTLZ5BNVH5F
CLIENT_SECRET:DSMRBBWX231WHXZZADRI2D4RYT1TDS033XEM11B2RIZWKWKG


In [34]:
# using the function from the hands on lab to get relevant information about venues in Manhattan
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']

        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [36]:
brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,49,49,49,49,49,49
Bay Ridge,89,89,89,89,89,89
Bedford Stuyvesant,28,28,28,28,28,28
Bensonhurst,32,32,32,32,32,32
Bergen Beach,7,7,7,7,7,7
Boerum Hill,82,82,82,82,82,82
Borough Park,18,18,18,18,18,18
Brighton Beach,43,43,43,43,43,43
Broadway Junction,15,15,15,15,15,15


In [37]:
#The one hot encoding is a good way to find out more information about the neighbourhoods
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head(20)

,Yoga Studio,Adult Boutique,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hockey Field,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video 

In [38]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped

,Neighborhood,Yoga Studio,Adult Boutique,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hockey Field,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video 

In [39]:
#Lets get the top 10 most common venues
num_top_venues = 10

for hood in brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                  venue  freq
0    Chinese Restaurant  0.06
1              Pharmacy  0.06
2       Bubble Tea Shop  0.04
3      Sushi Restaurant  0.04
4  Fast Food Restaurant  0.04
5    Italian Restaurant  0.04
6            Kids Store  0.04
7          Burger Joint  0.02
8                  Café  0.02
9  Cantonese Restaurant  0.02


----Bay Ridge----
                 venue  freq
0   Italian Restaurant  0.07
1                  Spa  0.07
2          Pizza Place  0.04
3  American Restaurant  0.03
4     Greek Restaurant  0.03
5   Chinese Restaurant  0.03
6           Bagel Shop  0.03
7             Pharmacy  0.03
8                  Bar  0.03
9       Sandwich Place  0.02


----Bedford Stuyvesant----
                 venue  freq
0          Coffee Shop  0.11
1                  Bar  0.07
2                 Café  0.07
3          Pizza Place  0.07
4             Bus Stop  0.07
5          Bus Station  0.04
6    Other Repair Shop  0.04
7         Cocktail Bar  0.04
8  Fried Chicken Joint

In [58]:
# The next step is to put this data in a dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Chinese Restaurant,Pharmacy,Bubble Tea Shop,Italian Restaurant,Kids Store,Sushi Restaurant,Fast Food Restaurant,Coffee Shop,Clothing Store,Rental Car Location
1,Bay Ridge,Spa,Italian Restaurant,Pizza Place,American Restaurant,Greek Restaurant,Bar,Pharmacy,Chinese Restaurant,Bagel Shop,Playground
2,Bedford Stuyvesant,Coffee Shop,Bus Stop,Pizza Place,Bar,Café,Other Repair Shop,Cocktail Bar,Gourmet Shop,Boutique,Bus Station
3,Bensonhurst,Chinese Restaurant,Sushi Restaurant,Ice Cream Shop,Donut Shop,Pizza Place,Italian Restaurant,Dumpling Restaurant,Liquor Store,Supermarket,Spa
4,Bergen Beach,Harbor / Marina,Athletics & Sports,Playground,Baseball Field,Donut Shop,Hockey Field,Flea Market,Fast Food Restaurant,Field,Filipino Restaurant


### k-Means Clustering

Finally I will cluster this data in 5 cluster and decide which cluster fits best to open up a typical German café.

In [59]:
from sklearn.cluster import KMeans
kclusters = 5

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

kmeans.labels_[0:10] 

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged = brooklyn_data

brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,2,Spa,Italian Restaurant,Pizza Place,American Restaurant,Greek Restaurant,Bar,Pharmacy,Chinese Restaurant,Bagel Shop,Playground
1,Brooklyn,Bensonhurst,40.611009,-73.995180,2,Chinese Restaurant,Sushi Restaurant,Ice Cream Shop,Donut Shop,Pizza Place,Italian Restaurant,Dumpling Restaurant,Liquor Store,Supermarket,Spa
2,Brooklyn,Sunset Park,40.645103,-74.010316,0,Bank,Pizza Place,Mexican Restaurant,Latin American Restaurant,Bakery,Pharmacy,Mobile Phone Shop,Gym,Deli / Bodega,Donut Shop
3,Brooklyn,Greenpoint,40.730201,-73.954241,2,Bar,Pizza Place,Coffee Shop,Cocktail Bar,Café,Bakery,French Restaurant,Mexican Restaurant,Record Shop,Boutique
4,Brooklyn,Gravesend,40.595260,-73.973471,0,Pizza Place,Lounge,Chinese Restaurant,Bus Station,Bakery,Martial Arts Dojo,Italian Restaurant,Sporting Goods Shop,Gym,Bar


In [60]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Discovering each cluster

In this last step I will examine each cluster to find out if there is on specific cluster with many Cafés, bakeries and German restaurants.

In [62]:
#Cluster 1
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Sunset Park,Bank,Pizza Place,Mexican Restaurant,Latin American Restaurant,Bakery,Pharmacy,Mobile Phone Shop,Gym,Deli / Bodega,Donut Shop
4,Gravesend,Pizza Place,Lounge,Chinese Restaurant,Bus Station,Bakery,Martial Arts Dojo,Italian Restaurant,Sporting Goods Shop,Gym,Bar
7,Manhattan Terrace,Pizza Place,Deli / Bodega,Cosmetics Shop,Grocery Store,Bank,Café,Mobile Phone Shop,Metro Station,Steakhouse,Chinese Restaurant
31,Manhattan Beach,Café,Bus Stop,Sandwich Place,Harbor / Marina,Food,Playground,Ice Cream Shop,Pizza Place,Beach,Food & Drink Shop
39,Sea Gate,Spa,Beach,Toy / Game Store,Bus Station,Video Store,Business Service,Optical Shop,Fish Market,Fast Food Restaurant,Field
46,Midwood,Pizza Place,Video Game Store,Pharmacy,Convenience Store,Falafel Restaurant,Candy Store,Bakery,Bagel Shop,Ice Cream Shop,Italian Restaurant
60,Mill Basin,Pizza Place,Japanese Restaurant,Burger Joint,Cosmetics Shop,Bank,Bagel Shop,Chinese Restaurant,Italian Restaurant,Sushi Restaurant,Restaurant
68,Madison,Bagel Shop,Dessert Shop,Deli / Bodega,Pizza Place,Italian Restaurant,Pilates Studio,Bus Station,Food,Flea Market,Fast Food Restaurant


In [63]:
#Cluster 2
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Canarsie,Thai Restaurant,Gym,Bus Line,Grocery Store,Food,Caribbean Restaurant,Chinese Restaurant,Asian Restaurant,Fish & Chips Shop,Field
59,Paerdegat Basin,Chinese Restaurant,Gym,Bus Line,Food,Asian Restaurant,Flea Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop


In [64]:
#Cluster 3
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 2, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Spa,Italian Restaurant,Pizza Place,American Restaurant,Greek Restaurant,Bar,Pharmacy,Chinese Restaurant,Bagel Shop,Playground
1,Bensonhurst,Chinese Restaurant,Sushi Restaurant,Ice Cream Shop,Donut Shop,Pizza Place,Italian Restaurant,Dumpling Restaurant,Liquor Store,Supermarket,Spa
3,Greenpoint,Bar,Pizza Place,Coffee Shop,Cocktail Bar,Café,Bakery,French Restaurant,Mexican Restaurant,Record Shop,Boutique
5,Brighton Beach,Beach,Eastern European Restaurant,Restaurant,Pharmacy,Fast Food Restaurant,Gourmet Shop,Mobile Phone Shop,Bank,Sushi Restaurant,Russian Restaurant
6,Sheepshead Bay,Dessert Shop,Turkish Restaurant,Sandwich Place,Pizza Place,Russian Restaurant,Restaurant,Creperie,Department Store,Yoga Studio,Buffet
9,Crown Heights,Pizza Place,Moving Target,Café,Museum,Deli / Bodega,Bagel Shop,Farmers Market,Liquor Store,Coffee Shop,Grocery Store
10,East Flatbush,Wine Shop,Department Store,Deli / Bodega,Moving Target,Pharmacy,Print Shop,Caribbean Restaurant,Chinese Restaurant,Park,Supermarket
11,Kensington,Thai Restaurant,Grocery Store,Ice Cream Shop,Sandwich Place,Pizza Place,Outdoors & Recreation,Mexican Restaurant,Supermarket,Furniture / Home Store,Lingerie Store
12,Windsor Terrace,Diner,Park,Food Truck,Plaza,Middle Eastern Restaurant,Bakery,Bookstore,Grocery Store,Sushi Restaurant,Beer Store
13,Prospect Heights,Bar,Thai Restaurant,Mexican Restaurant,Coffee Shop,Gourmet Shop,Cocktail Bar,Bakery,Wine Shop,Beer Bar,Pizza Place


In [65]:
#Cluster 4
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Mill Island,Pool,Lake,Food,Other Repair Shop,Discount Store,Distillery,French Restaurant,Food Truck,Food Stand,Food Court


In [66]:
#Cluster 5
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 4, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Flatbush,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Mexican Restaurant,Bank,Middle Eastern Restaurant,Hardware Store,Plaza,Pharmacy,Deli / Bodega
26,East New York,Fast Food Restaurant,Plaza,Pizza Place,Liquor Store,Deli / Bodega,Bus Station,Spanish Restaurant,Gym,Metro Station,Asian Restaurant
27,Starrett City,Convenience Store,Donut Shop,Liquor Store,Gym,Supermarket,Shoe Store,Cosmetics Shop,Caribbean Restaurant,Shopping Mall,Pharmacy
29,Flatlands,Pharmacy,Fast Food Restaurant,Discount Store,Caribbean Restaurant,Deli / Bodega,Paper / Office Supplies Store,Fried Chicken Joint,Nightclub,Lounge,Sandwich Place
32,Coney Island,Baseball Stadium,Caribbean Restaurant,Monument / Landmark,Café,Gourmet Shop,Theme Park Ride / Attraction,Farmers Market,Music Venue,Brewery,Pharmacy
34,Borough Park,Pizza Place,Bank,Pharmacy,Deli / Bodega,Hotel,American Restaurant,Bike Shop,Fast Food Restaurant,Coffee Shop,Cheese Shop
47,Prospect Park South,Caribbean Restaurant,Pizza Place,Fast Food Restaurant,Grocery Store,Pharmacy,Mobile Phone Shop,Donut Shop,Clothing Store,Latin American Restaurant,Supermarket
55,Wingate,Fast Food Restaurant,Deli / Bodega,Gym / Fitness Center,BBQ Joint,Bus Station,Fried Chicken Joint,Garden Center,Fish & Chips Shop,Field,Sandwich Place
56,Rugby,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Ice Cream Shop,Diner,Mobile Phone Shop,Pharmacy,Pizza Place,Fried Chicken Joint,Fast Food Restaurant
57,Remsen Village,Fast Food Restaurant,Caribbean Restaurant,Fried Chicken Joint,Fish Market,Salad Place,Gym,Café,Coffee Shop,Supermarket,Donut Shop


### Final Conclusion

After getting, filtering, sorting and clustering the location data from Brooklyn to find out if there is a cluster which contains many cafés, bakeries and German restaurants I came to the following conclusion: cluster 1 and cluster 3 are focusing on food locations and therefore also cafés, bakeries and many more international food stores. Unfortunately, there was no neighbourhood with a German restaurant under the top ten venues. Nevertheless, cluster 1 and cluster 3 contain about 80% of all neighbourhoods in Brooklyn. This means that there is not this one place with is best for opening a typical German Café. Therefore: try and error :-)